In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

In [2]:
CHROME_PATH = "/Users/ik/bin/chromedriver"

In [3]:
# initialize web driver with eager page loading strategy
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
driver = webdriver.Chrome(CHROME_PATH, options=chrome_options)

## Crawl main index site for list of links to each state/regions branch directory

In [ ]:
# get links to each region home page
driver.get('https://locations.blinkfitness.com/index.html')
links = WebDriverWait(driver, timeout=30).until(lambda d: d.find_elements_by_tag_name('a'))

location_urls = [] # will contain final urls
skip = ['index','search']

for link in links:
    # get all links on webpage
    url_string = link.get_attribute('href')
    
    # check for branch state sublinks only 
    if url_string.startswith('https://locations.blinkfitness.com/'):
        if not any(x in url_string for x in skip):
            branch_state = link.text 
            location_urls.append(url_string)

# will handle special case separately
location_urls.remove('https://locations.blinkfitness.com/va/virginia-beach/4239-holland-road')

## Parse individual branches from each region directory page

In [ ]:
location_info = []

for region_url in location_urls:
    driver.get(region_url)
    
    # parse city containers
    cities = WebDriverWait(driver, timeout=30).until(lambda d: d.find_elements_by_class_name('Directory-cityContainer'))  
    
    for city in cities:
        branch_city = city.find_element_by_class_name('Directory-cityName').text
        
        # get list of all branches in city container
        branches = city.find_elements_by_class_name('Directory-listTeaser')
        
        # parse each branch for title, address & phone number
        for branch in branches:
            temp_branch = {}
            # temp_branch['branch_state'] = 
            temp_branch['branch_city'] = branch_city
            temp_branch['branch_title'] = branch.find_element_by_class_name('Teaser-title').text
            temp_branch['branch_address'] = branch.find_element_by_class_name('Teaser-address').text
            temp_branch['branch_phone'] = branch.find_element_by_class_name('Teaser-phone').text
            
            location_info.append(temp_branch)

## Handle parsing of special case: Virginia Beach branch
Virginia only has 1 location right now so it doesn't have a separate directory page like all other states

In [ ]:
temp = {}
va_url = 'https://locations.blinkfitness.com/va/virginia-beach/4239-holland-road'

driver.get(va_url)
va_info = WebDriverWait(driver, timeout=30).until(lambda d: d.find_element_by_class_name('Core'))  

temp['branch_state'] = 'Virginia'
temp['branch_city'] = 'Virginia Beach'
temp['branch_title'] = va_info.find_element_by_class_name('LocationName-geo').text
temp['branch_phone'] = va_info.find_element_by_id('phone-main').text
temp['branch_address'] = va_info.find_element_by_class_name('c-address').text

location_info.append(temp_info)

In [ ]:
location_info

In [ ]:
for i in location_info:
    print(i)